In [4]:
import pandas as pd
import datetime
import requests
import tabula
import io
import os
import sys

#URLlib
import urllib
import urllib.request

#PDFminer
import pdfminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal

In [20]:
#Ojo Cambiar el nombre del archivo
path = "statement.pdf"

In [ ]:
key_words = {"menor volatilidad",
            "mayor volatilidad",
            "perspectivas crediticias",
            "holgura",
            "balance",
            "inflación",
            "inflación subyacente",
            "finanzas públicas",
            "incertidumbre",
            "postura monetaria prudente"
            "actividad económica",
            "economía",
            "balance"}

In [21]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [22]:
pdf_text = convert_pdf_to_txt(path).replace('\n', '')

In [23]:
words_to_replace = ["\x0c", ".", ",", " i)", "ii", "iii", "iv)", "v)", "vi)", ")", " i ", " v ", " vi ", " iv "]
for word in words_to_replace:
    pdf_text = pdf_text.replace(word, "")
pdf_text = pdf_text.replace("  ", " ")


In [27]:
words_df = pd.DataFrame(columns=['Count'])
for phrase in key_words:
    words_df.loc[phrase, "Count"] = pdf_text.count(phrase)

words_df


,Count
inflación subyacente,3.0
inflación,17.0
balance,3.0
finanzas públicas,2.0
perspectivas crediticias,2.0
menor volatilidad,1.0
mayor volatilidad,0.0
economía,10.0
holgura,5.0
postura monetaria prudenteactividad económica,0.0
